In [ ]:
# this code makes a call to the spotify API and gets the all the albums for each artist in the list and saves them to a JSON file
import os
import json
import requests
import time

def load_artist_ids(file_path):
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        return []

def get_artist_albums(artist_id, access_token, max_requests=10, max_time=60):
    url = f'https://api.spotify.com/v1/artists/{artist_id}/albums'
    headers = {'Authorization': f'Bearer {access_token}'}
    params = {'limit': 50}
    albums = []
    start_time = time.time()
    while len(albums) < max_requests and time.time() - start_time < max_time:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            albums.extend(data['items'])
            total = data['total']
            params['offset'] = len(albums)
            if len(albums) >= total:
                break
        else:
            print(f"Failed to get albums for {artist_id}. Status code: {response.status_code}")
            print(response.headers)
            return []
    return albums

# Replace 'YOUR_ACCESS_TOKEN' with your actual Spotify API access token and the path to the file containing artist IDs and the output directory
artist_ids_file_path = 'relatedartistslist.txt'
output_directory = 'artist_albums'
access_token = 'REPLACE_WITH_YOUR_ACCESS_TOKEN'

os.makedirs(output_directory, exist_ok=True)
artist_ids = load_artist_ids(artist_ids_file_path)
for artist_id in artist_ids:
    artist_albums = get_artist_albums(artist_id, access_token)
    with open(f'{output_directory}/{artist_id}.json', 'w') as output_file:
        json.dump(artist_albums, output_file)
print("Artist albums data has been saved to", output_directory)